# 1 - Listado de variables y selección

## Entrada
* Manufacturer: Marca / fabricante del equipo
* Category: Tipo de notebook
* Screen Size: Tamaño diagonal de la pantalla en pulgadas
* Screen Type: Resolución y tipo de panel
* CPU: Modelo de procesador
* RAM: Capacidad de memoria
* Storage: Tipo y tamaño de almacenamiento interno
* GPU: Tipo y modelo de procesador de gráficos
* OS: Sistema operativo
* Weight: Peso

## Salida
* Price: Será reducida con rangos a valores "bajo", "medio" o "alto"

## No utilizadas
* Model Name: Texto arbitrario que generalmente no representa las capacidades del producto, y cuando lo hace meramente tiene agregados valores de otras columnas
* OS Version: No usada debido a su gran cantidad de nulos y presunta poca importancia

# 2 - Análisis detallado de un conjunto de variables

In [ ]:
import pandas as pd

In [ ]:
laptops_train = pd.read_csv('laptops_train.csv')
laptops_test = pd.read_csv('laptops_test.csv')

In [ ]:
print("Entries de train: "+str(len(laptops_train)))
print("Entries de test: "+str(len(laptops_test)))

### Valores nulos

In [ ]:
laptops_train.isnull().sum()

In [ ]:
laptops_test.isnull().sum()

Este dataset solo presenta valores nulos en la columna "Operating System Version", la cual se decidió no utilizar anteriormente, por lo cual no es un problema a tener en cuenta en este caso

In [ ]:
laptops_train

In [ ]:
# Renombre de las series
new_col_names = {
    "Manufacturer":"manufacturer",
    "Model Name":"model",
    "Category":"category",
    "Screen Size":"screen_size",
    "Screen":"display",
    "CPU":"cpu",
    "RAM":"gb_ram",
    " Storage":"storage", # la string original tiene el espacio, sin eso no lo cambia
    "GPU":"gpu",
    "Operating System":"os",
    "Operating System Version":"os_version",
    "Weight":"weight",
    "Price":"price"
}

dropped_cols = ["os_version", "model"]

laptops_train = laptops_train.rename(columns=new_col_names).drop(columns=dropped_cols)
laptops_test = laptops_test.rename(columns=new_col_names).drop(columns=dropped_cols)

In [ ]:
laptops_train

## Variable de salida

In [ ]:
# Conversión de la variable de salida de rupias a dólares, tomando como base la cotización del 28/03/2023, según la última actualización del dataset
# Se dividió por 100 el valor obtenido debido a que los precios convertidos con la cotización de referencia se encontraban fuera del rango esperado
# Cotización: 1 INR = 0.0001217 USD


cotizacion = 0.0001217

def convertir_precio(precio):
    return int(precio * cotizacion)

laptops_train["price"] = laptops_train["price"].apply(convertir_precio)
laptops_test["price"] = laptops_test["price"].apply(convertir_precio)

In [ ]:
laptops_train

In [ ]:
laptops_test

In [ ]:
import matplotlib as plt
import plotly.express as px

In [ ]:
px.histogram(laptops_train, x="price")

In [ ]:
px.histogram(laptops_test, x="price")

In [ ]:
px.box(laptops_train, x="price")

In [ ]:
px.box(laptops_test, x="price")

In [ ]:
laptops_train["price"].describe()

In [ ]:
laptops_test["price"].describe()

Se puede apreciar como la variable de salida tiene una forma de campana gaussiana con asimetría positiva. Podemos notar como tanto el set de train y de test tienen una distribución similar, lo cual es importante controlar debido a que ambas tablas fueron provistas por el creador del dataset. A su vez se puede ver como existen valores anómalos y aberrantes en los rangos superiores del dataset. En consecuencia, esto podría generar overfitting debido a los pocos datos en este rango, y optamos por quitar estos dispositivos del dataset directamente.

In [ ]:
# Descartamos los valores anómalos y aberrantes (donde precio > 3500)

descartes_train = laptops_train[laptops_train.price > 3500].index
descartes_test = laptops_test[laptops_test.price > 3500].index

laptops_train = laptops_train.drop(descartes_train)
laptops_test = laptops_test.drop(descartes_test)

print("Valores descartados:")
print("Train:"+str(len(descartes_train)))
print("Test:"+str(len(descartes_test)))

In [ ]:
# Conversión a rangos de precios
# Rangos:
# low: 0 < price <= 1000
# mid: 1000 < price <= 1500
# high: 1500 < price <= 3500

laptops_train["price_range"] = pd.cut(x=laptops_train["price"], bins=[0, 1000, 1500, 3500], labels=["low", "mid", "high"])
laptops_test["price_range"] = pd.cut(x=laptops_test["price"], bins=[0, 1000, 1500, 3500], labels=["low", "mid", "high"])

In [ ]:
# Ordenamos el dataset por la nueva columna de price_range para que los gráficos queden mejor acomodados
# (se puede hacer agregando category_orders={"price_range": ["low", "mid", "high"]} a cada gráfico, pero de esta manera aplica para todos)

laptops_train = laptops_train.sort_values(by="price_range")
laptops_test = laptops_test.sort_values(by="price_range")

In [ ]:
laptops_train

In [ ]:
#Una ves aplicado los rangos, droppeamos la columna de precio, porque no la vamos a usar
laptops_train = laptops_train.drop(columns="price")
laptops_test = laptops_test.drop(columns="price")

In [ ]:
laptops_train

In [ ]:
px.histogram(laptops_train, x = "price_range")

## Variables de entrada

### Manufacturer

In [ ]:
px.histogram(laptops_train,x="manufacturer", color="price_range").update_xaxes(categoryorder="sum descending")

Se puede observar como el dataset cuenta en su mayoria con 4 manufacturadores predominantes, Lenovo, Dell, HP y Asus.

### Category

In [ ]:
px.histogram(laptops_train, x="category", color="price_range").update_xaxes(categoryorder="sum descending")

Se observa como la mayoria de los datos ingresados tienen como categoria "Notebook". Ademas, la categoria "Workstation" contiene solo valores donde el rango de precio es "High" lo cual influira en la decision tomada por el modelo. Se puede notar tambien como las categorias "Gaming" y "Ultrabook" cuentan con muchos valores "mid" y "high", lo que tambien influye en las decisiones finales.

### Screen size

In [ ]:
px.histogram(laptops_train, x="screen_size", color="price_range").update_xaxes(categoryorder="category ascending")

Se puede ver como la mayoria de los datos cuentan con un tamaño de pantalla de 15,6 pulgadas. Tambien se puede notar algunos datos individuales esparcidos entre los otros tamaños. A esta columna le aplicaremos una conversión de string a float, y dividimos arbitrariamente por 10 su valor

In [ ]:
# Conversión de string a float

def convertir_tamanio_pantalla(tamanio):
    return float(tamanio[:-1])/10

laptops_train["screen_size"] = laptops_train["screen_size"].apply(convertir_tamanio_pantalla)
laptops_test["screen_size"] = laptops_test["screen_size"].apply(convertir_tamanio_pantalla)

In [ ]:
px.histogram(laptops_train, x="screen_size", color="price_range")

### Display

In [ ]:
px.histogram(laptops_train, x="display", color="price_range", height=600)

Esta columna posee valores conflictivos, debido a que varios términos representan lo mismo, y que estas strings no son útiles a la hora de entrenar un modelo, por lo cual esta columna será subdividida en 3:
* panel_res_x: resolución en el eje X
* panel_res_y: resolución en el eje Y
* panel_tech: lista de términos de features como "IPS" o "Touchscreen", usada luego como entrada de un count vectorizer

In [ ]:
def obtener_pixeles_resolucion(display_string, axis):
    res_array = display_string.split()
    res = [substring for substring in res_array if "x" in substring][0]
    axis = 0 if axis == "x" else 1
    return int(res.split('x')[axis])

In [ ]:
# Creación de la columna res_x

laptops_train.insert(4, "panel_res_x", laptops_train["display"].apply(obtener_pixeles_resolucion, args=("x")))
laptops_test.insert(4, "panel_res_x", laptops_test["display"].apply(obtener_pixeles_resolucion, args=("x")))

In [ ]:
# Creación de la columna res_y

laptops_train.insert(5, "panel_res_y", laptops_train["display"].apply(obtener_pixeles_resolucion, args=("y")))
laptops_test.insert(5, "panel_res_y", laptops_test["display"].apply(obtener_pixeles_resolucion, args=("y")))

In [ ]:
# Código auxiliar para detectar las posibles features de los displays del dataset

features = set()

for display in laptops_train["display"]:
     for word in display.split():
         features.add(word)

print(features)

In [ ]:
# Creación de la columna panel_tech

# features consideradas relevantes, dado que las resoluciones tienen sus columnas separadas y variantes de "HD" representan esos mismos números
features = ["IPS", "Touchscreen", "Retina"]

def obtener_features_panel(panel):
    panel_strings = panel.split()
    return " ".join([feature for feature in panel_strings if feature in features])

laptops_train.insert(6, "panel_tech", laptops_train["display"].apply(obtener_features_panel))
laptops_test.insert(6, "panel_tech", laptops_test["display"].apply(obtener_features_panel))

In [ ]:
# Se elimina la columna "display" debido a que su información relevante ya fue transferida a las nuevas columnas

laptops_train = laptops_train.drop(columns="display")
laptops_test = laptops_test.drop(columns="display")

In [ ]:
laptops_train

In [ ]:
px.histogram(laptops_train, x="panel_res_x", color="price_range", height=600)

In [ ]:
px.histogram(laptops_train, x="panel_res_y", color="price_range", height=400)

In [ ]:
px.scatter_3d(laptops_train, x="panel_res_x", y="panel_res_y", z="price_range", color="price_range", height=600)

In [ ]:
px.histogram(laptops_train, x="panel_tech", color="price_range", height=600)

### CPU

In [ ]:
px.histogram(laptops_train, x="cpu", color="price_range", height=700)

Se nota como la variable con mas ejemplares es la de Intel Core i5, y como el rango de precios aumenta con CPUs mas modernos

### RAM

In [ ]:
px.histogram(laptops_train, x="gb_ram", color="price_range", barmode="group", category_orders={"gb_ram": ["2GB", "4GB", "6GB", "8GB", "12GB", "16GB", "24GB", "32GB"]})

Se puede apreciar como los rangos de precios ocupan mayoritariamente su segmento correspondiente, es decir, la mayoría de laptops con poca RAM se encuentran en el rango de precios bajos, aquellas con una cantidad media en el rango de precio medio, y las que tienen más capacidad de memoria en el rango de precio alto. Dado que todas las capacidades de memoria están listadas en GB, se pueden convertir a integer tranquilamente

In [ ]:
# Conversión de RAM a integere

def convertir_ram_a_int(capacidad):
    return(int(capacidad[:-2]))

laptops_train["gb_ram"] = laptops_train["gb_ram"].apply(convertir_ram_a_int)
laptops_test["gb_ram"] = laptops_test["gb_ram"].apply(convertir_ram_a_int)

In [ ]:
px.histogram(laptops_train, x="gb_ram", barmode="group", color="price_range")

### Storage

In [ ]:
px.histogram(laptops_train, x="storage", color="price_range", height=600)

Para esta variable, dividiremos los GB de SSD y HDD en columnas separadas. Se eliminarán las entries que tengan almacenamiento listado como "Hybrid" ya que no se puede saber la cantidad de GB de cada medio. A su vez, los listados como "Flash Storage" serán tomados como GB de SSD.

In [ ]:
# Quitamos las computadoras con almacenamiento "Híbrido"

descartes_train = laptops_train[laptops_train.storage.str.contains("Hybrid")].index
descartes_test = laptops_test[laptops_test.storage.str.contains("Hybrid")].index

laptops_train = laptops_train.drop(descartes_train)
laptops_test = laptops_test.drop(descartes_test)

print("Valores descartados:")
print("Train:"+str(len(descartes_train)))
print("Test:"+str(len(descartes_test)))

In [ ]:
# Convertimos "Flash Storage" a "SSD"

def convertir_a_ssd(storage_string):
    return storage_string.replace("Flash Storage", "SSD")

laptops_train["storage"] = laptops_train["storage"].apply(convertir_a_ssd)
laptops_test["storage"] = laptops_test["storage"].apply(convertir_a_ssd)

In [ ]:
# Función para obtener los GB de SSD y HDD

def separar_unidades_almacenamiento(storage_string, medio):
    if "+" in storage_string:
        storage_string = " ".join(storage_string.split("+"))
    storage_array = storage_string.split()
    
    if medio == "S" and "SSD" in storage_array:
        ssd = 0
        for index, string in enumerate(storage_array):
            if string == "SSD":
                drive_capacity = storage_array[index-1]
                multiplier = 1
                if "TB" in drive_capacity:
                    multiplier = 1024
                ssd += int(drive_capacity.strip('GTB ')) * multiplier
        return ssd
    
    if medio == "H" and "HDD" in storage_array:
        hdd = 0
        for index, string in enumerate(storage_array):
            if string == "HDD":
                drive_capacity = storage_array[index-1]
                multiplier = 1
                if "TB" in drive_capacity:
                    multiplier = 1024
                hdd += int(drive_capacity.strip('GTB ')) * multiplier
        return hdd
    return 0

In [ ]:
# Creación de la columna gb_ssd

laptops_train.insert(9, "gb_ssd", laptops_train["storage"].apply(separar_unidades_almacenamiento, args=("S")))
laptops_test.insert(9, "gb_ssd", laptops_test["storage"].apply(separar_unidades_almacenamiento, args=("S")))

In [ ]:
# Creación de la columna gb_hdd

laptops_train.insert(10, "gb_hdd", laptops_train["storage"].apply(separar_unidades_almacenamiento, args=("H")))
laptops_test.insert(10, "gb_hdd", laptops_test["storage"].apply(separar_unidades_almacenamiento, args=("H")))

In [ ]:
# Se elimina la columna "storage" debido a que su información relevante ya fue transferida a las nuevas columnas

laptops_train = laptops_train.drop(columns="storage")
laptops_test = laptops_test.drop(columns="storage")

In [ ]:
laptops_train

In [ ]:
px.scatter_3d(laptops_train, x="gb_ssd", y="gb_hdd", z="price_range", color="price_range")

### GPU

In [ ]:
px.histogram(laptops_train, x="gpu", color="price_range", height=600)

Las GPU mas caras resultan ser las pertenecientes a NVidia, aumentando el precio con las gpu mas modernas. La mas comun en el dataset son las de Intel HD Graphics.

### OS

In [ ]:
px.histogram(laptops_train, x="os", color="price_range")

La gran mayoria de las laptops tiene como sistema operativo a Windows. Se aprecia una discrepancia al haber dos columnas de mac OS, lo cual resolvemos con la siguiente operación

In [ ]:
# Normalización de nombres de macOS

def acomodar_nombre_os(nombre):
    if nombre == "Mac OS":
        nombre = "macOS"
    return nombre

laptops_train["os"] = laptops_train["os"].apply(acomodar_nombre_os)
laptops_test["os"] = laptops_test["os"].apply(acomodar_nombre_os)

In [ ]:
px.histogram(laptops_train, x="os", color="price_range")

### Weight

In [ ]:
px.histogram(laptops_train, x="weight", color="price_range").update_xaxes(categoryorder="category ascending")

En cuanto al peso, se puede notar como la mayoria de los equipos pesan 2.2kg, marcando su precio como "bajo".
Observando la grafica podemos ver como el precio aumenta extremos, manteniendose en un precio bajo en los pesos intermedios

In [ ]:
# Conversión de pesos a floats

def convertir_peso(peso):
    return float(peso.strip('kgs '))

laptops_train["weight"] = laptops_train["weight"].apply(convertir_peso)
laptops_test["weight"] = laptops_test["weight"].apply(convertir_peso)

In [ ]:
px.histogram(laptops_train, x="weight", color="price_range")

## Listado de dudas/preguntas para el proveedor del dataset

* De que fuentes provienen estos precios? y que metodos fueron utilizados para recolectarlos?
* Porque la columna de SO Type tiene tantos valores nulos?
* Pensas que el tipo de SO afecta al precio?
* Pensas que existen otros factores que afecten el precio de una laptop? como serían el material de chasis, distribución de teclado, etc.
* Cual crees que es la variable que mas afecta al precio de una laptop?
* Omitiste alguna variable al creear el dataset?

## 3 - Hipotesis sobre los datos

* Pensamos que la variable OS influye poco sobre el rango de precio final
* Creemos que las variables mas influyentes son GPU, Storage, RAM y CPU
* La variable Category se ve afectada principalmente por el GPU? 

## Comprobacion de la hipotesis

## Creacion de nuevas variables

## 4 - Modelado